In [8]:
import os
import pandas as pd
import numpy as np

In [261]:
path = './all-census-data/'
#read literacy data into dataframe
literacy_data = pd.read_csv(path + 'elementary_2015_16.csv', usecols=list(range(1,9)) + list(range(11,17)))

In [262]:
#convert x females per 1000 males to a decimal (female to 1 male)
# if greater than to avoid running multiple times
if (literacy_data['SEX RATIO'].mean() > 1):
    literacy_data['SEX RATIO'] = literacy_data['SEX RATIO']/1000

In [263]:
#drop rows with null values
literacy_data.dropna(inplace=True)

In [272]:
#group dataframe by 'STATE NAME'
groupedby_state = literacy_data.groupby('STATE NAME')

In [291]:
#aggregate function to call on grouped DataFrame to return weight literacy rates (weighted by respective population size)
def lit_agg(x):
    names = {
        'WEIGHTED MALE LITERACY' : np.average(x['MALE LITERACY'], weights=(literacy_data.loc[x.index, "TOTAL POULATION"]) - (literacy_data.loc[x.index, "SEX RATIO"] * literacy_data.loc[x.index, "TOTAL POULATION"])),
        'WEIGHTED FEMALE LITERACY' : np.average(x['FEMALE LITERACY'], weights=(literacy_data.loc[x.index, "SEX RATIO"] * literacy_data.loc[x.index, "TOTAL POULATION"])),
        'WEIGHTED TOTAL LITERACY' : np.average(x['OVERALL LITERACY'], weights=(literacy_data.loc[x.index, "TOTAL POULATION"])),
        'WEIGHTED SEX RATIO': np.average(x['SEX RATIO'], weights=(literacy_data.loc[x.index, "TOTAL POULATION"])), 
        'WEIGHTED URBAN POPULATION': np.average(x['PERCENTAGE URBAN POPULATION'], weights=(literacy_data.loc[x.index, "TOTAL POULATION"]))
        }
    return pd.Series(names, index=['WEIGHTED MALE LITERACY', 'WEIGHTED FEMALE LITERACY', 'WEIGHTED TOTAL LITERACY', 'WEIGHTED SEX RATIO', 'WEIGHTED URBAN POPULATION'])


In [305]:
#.apply aggregate function
weighted_census_data = groupedby_state.apply(lit_agg)

#bin percentage of urban population in 0-33%, 33-66%, 66-100%
bins = [0, 33, 66, 100]
weighted_census_data['URBAN POPULATION BIN'] = pd.cut(weighted_census_data['WEIGHTED URBAN POPULATION'], bins)

In [121]:
path = './all-census-data/'
 
files = os.listdir(path)

In [6]:
gdp_files = [file for file in files if 'gdp' in file]